In [0]:
#all imports
import tensorflow as tf
import torch

In [2]:

gpuname=tf.test.gpu_device_name()
if gpuname=='/device:GPU:0':
  print('Found GPU at :{}'.format(gpuname))
else:
  raise(SystemError('GPU device not found'))

Found GPU at :/device:GPU:0


#Custom Data set and Data loader has  been adapted and inspired from 
#Michael Sugimura,Github Repository:https://github.com/sugi-chan/custom_bert_pipeline
#BERT based fine tuning adapted and inspired from:Chris McCormick and Nick Ryan. (2019, July 22). BERT Fine-Tuning #Tutorial with PyTorch. Retrieved from http://www.mccormickml.com
#for all references refer README.md

In [3]:
if torch.cuda.is_available():
  device=torch.device("cuda")
  print("There are %d GPU DEVICES available " %torch.cuda.device_count())
  print("The device name is %s"%torch.cuda.get_device_name(0))
else:
  print("No GPU available using only CPU instead")
  device=torch.device("cpu")



There are 1 GPU DEVICES available 
The device name is Tesla P100-PCIE-16GB


In [4]:
!pip install transformers

     |████████████████████████████████| 675kB 3.5MB/s 
     |████████████████████████████████| 3.8MB 17.7MB/s 
     |████████████████████████████████| 890kB 43.6MB/s 
     |████████████████████████████████| 1.2MB 49.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=9637245559511fc13cade50c82859373e3672657ecae87879d586a6ba492b5f3
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!unzip -P ***** -qq '/content/drive/My Drive/DanishData/Danish.zip'

In [0]:
import csv
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [0]:
#GET THE DATA FROM THE PANDAS FRAME
headers=['id','tweet','subtask_a']
greekdata = pd.read_csv("Danish/offenseval-da-training-v1.tsv", delimiter='\t',names=headers)
data=greekdata[1:]


In [0]:
data=data[:2960]

In [10]:
data.to_numpy()[:,2]

array(['NOT', 'NOT', 'OFF', ..., 'NOT', 'NOT', 'OFF'], dtype=object)

In [0]:

dfnumpy=data.to_numpy();
x=dfnumpy[:, 1].reshape(-1, 1)
y=dfnumpy[:, 2].reshape(-1, 1)


In [0]:
import re

def ft1(l):
    a = re.split('|\]|\[|\)|\(|; |,|\*|\n',l)
    li = []
    for t in a:
        li.append(t.strip(' ,'))
    return li

In [0]:
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords 
import string
tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)
def preprocessinglib(arrt):
    ans =[]
    for txt in arrt:
        ans.append(ft1(txt))
               
    return ans


In [14]:
arrt=x[:,0]
allTokens=preprocessinglib(arrt)

/usr/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [0]:
preprocessedTweets=allTokens

In [0]:

from sklearn.utils import shuffle
preprocessedTweets, y = shuffle(preprocessedTweets, y)

In [17]:
y

array([['NOT'],
       ['NOT'],
       ['NOT'],
       ...,
       ['OFF'],
       ['NOT'],
       ['NOT']], dtype=object)

In [18]:
print(preprocessedTweets[0:5])

[['Cupcake!'], ['Og så sidder man her og æder ris og tun ..'], ['Jeg ved hvor du bor nu.  &#x200B;'], ['... Men svensken er jo til at se, desværre.'], ['###En smuk skibsnation']]


In [19]:
from transformers import BertTokenizer as bertTokenizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset,DataLoader,RandomSampler,SequentialSampler

Using TensorFlow backend.


In [20]:
tokenizer=bertTokenizer.from_pretrained('bert-base-multilingual-cased',do_lower_case=True)


In [0]:
ids_of_sentence=[]
ids_of_sentence_words=[]
attention_masks=[]
def giveIds(sentence,y_):
  ids_of_sentence=[]
  ids_of_sentence_words=[]
  attention_masks=[]
  maxlength=0
  for t in sentence:
      tokenized_sentence_id=tokenizer.encode(t,add_special_tokens=True)
      if(maxlength<len(tokenized_sentence_id)):
          maxlength=len(tokenized_sentence_id)
      ids_of_sentence.append(tokenized_sentence_id)
  print(maxlength)
  ids_of_sentence_words=pad_sequences(ids_of_sentence,maxlen=maxlength,dtype="long",value=0,truncating="post",padding="post")##can change max length
  attention_masks = [[int(a > 0)   for a in b ]for b in ids_of_sentence_words] 
  #print(len(attention_masks))
  #print(len(ids_of_sentence_words))
  return ids_of_sentence_words,attention_masks

In [0]:
sentence_train, sentence_test, y_train, y_test = train_test_split(preprocessedTweets,y, test_size=0.2, random_state=42)
#x_train_mask,x_test_mask,_,_=train_test_split(attention_masks,y, test_size=0.2, random_state=42)



In [23]:

from sklearn.svm import SVC
from sklearn import preprocessing

le = preprocessing.LabelEncoder()

print(y_train.shape)
print(y_test.shape)
yTrain=le.fit_transform(y_train.flatten())
print(yTrain.shape)
print(le.classes_)


(2368, 1)
(592, 1)
(2368,)
['NOT' 'OFF']


In [0]:
yTest=le.fit_transform(y_test.flatten())


In [25]:
set(y_test.flatten())

{'NOT', 'OFF'}

In [26]:
set(yTest)

{0, 1}

In [0]:
from torch.utils.data import Dataset
tokenizer=bertTokenizer.from_pretrained('bert-base-multilingual-cased',do_lower_case=True)
class GreekTrainDataset(Dataset):
    def __init__(self,xytrain):
        self.xytrain = xytrain
        self.maxlength=256
       
    def __getitem__(self, index):
        tokenized_review = tokenizer.tokenize(str(self.xytrain[0][index]))
        if len(tokenized_review) > self.maxlength:
            #print(tokenized_review)
            tokenized_review = tokenized_review[:self.maxlength]
        
        
        ids_of_sentence_word  = tokenizer.convert_tokens_to_ids(tokenized_review)
        padding = [0] * (self.maxlength - len(ids_of_sentence_word))
        ids_of_sentence_word += padding
        assert len(ids_of_sentence_word) == self.maxlength
        #print(ids_of_sentence_word)
        attention_mask = [int(b > 0) for b in ids_of_sentence_word] 
        x_train_pytorch = torch.tensor(ids_of_sentence_word)
        y_train_pytorch=torch.tensor(self.xytrain[1][index])
        x_train_mask_pytorch=torch.tensor(attention_mask)
        
        return x_train_pytorch,x_train_mask_pytorch,y_train_pytorch
        
        
    def __len__(self):
        return len(self.xytrain[0])

In [0]:
#from torch.utils.data import Dataset
#tokenizer=bertTokenizer.from_pretrained('bert-base-multilingual-cased',do_lower_case=True)
class GreekTestDataset(Dataset):
    def __init__(self,xytest):
        self.xytest = xytest
        self.maxlength=256
       
    def __getitem__(self, index):
        tokenized_review = tokenizer.tokenize(str(self.xytest[0][index]))
        if len(tokenized_review) > self.maxlength:
            #print(tokenized_review)
            tokenized_review = tokenized_review[:self.maxlength-2]
        
        
        ids_of_sentence_word  = tokenizer.convert_tokens_to_ids(tokenized_review)
        padding = [0] * (self.maxlength - len(ids_of_sentence_word))
        ids_of_sentence_word += padding
        assert len(ids_of_sentence_word) == self.maxlength
        #print(ids_of_sentence_word)
        attention_mask = [int(b > 0) for b in ids_of_sentence_word] 
        x_test_pytorch = torch.tensor(ids_of_sentence_word)
        y_test_pytorch=torch.tensor(self.xytest[1][index])
        x_test_mask_pytorch=torch.tensor(attention_mask)
        
        return x_test_pytorch,x_test_mask_pytorch,y_test_pytorch
        #return [1,2,3]
        
    def __len__(self):
        return len(self.xytest[0])

In [0]:
xytrain=[sentence_train,yTrain]
tdataset = GreekTrainDataset(xytrain)
tsampler=RandomSampler(tdataset)
tdataloader = DataLoader(tdataset, batch_size=32, num_workers=1, shuffle=False,sampler=tsampler)

In [0]:
xytest=[sentence_test,yTest]
tedataset = GreekTestDataset(xytest)
tesampler=RandomSampler(tedataset)
tedataloader = DataLoader(tedataset, batch_size=32, num_workers=1, shuffle=False,sampler=tesampler)

In [31]:
from transformers import BertForSequenceClassification as bfsc,AdamW,BertConfig
model=bfsc.from_pretrained('bert-base-multilingual-cased',num_labels=2,output_attentions=False,output_hidden_states=False)
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [32]:

torch.save({'state_dict': model.state_dict()}, '/content/drive/My Drive/DanishData/bertdanishlast.pth.tar')
checkpoint = torch.load('/content/drive/My Drive/DanishData/bertdanishlast.pth.tar')
model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [0]:
params=list(model.named_parameters())

In [0]:
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
{
"params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
"weight_decay": 0.01,
},
{"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
]

In [0]:
optimizer=AdamW(optimizer_grouped_parameters,lr=2e-5,eps=1e-8)
#############CAN CHANGE LEARNING RATE 

In [0]:
from transformers import get_linear_schedule_with_warmup

epochs=4
total_steps=len(tdataloader)*epochs

sch=get_linear_schedule_with_warmup(optimizer,
                                    num_warmup_steps=0,num_training_steps=total_steps)

In [0]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

def calculateF1Score(predictions,labels):
  #rowwise return the index of the max element ie 0 or 1 depending on the maximum value returned
  predictionArgmax=np.argmax(predictions,axis=1).flatten()
  labelsFlattend=labels.flatten()
  print("predictionArgmax",predictionArgmax)
  print("labelsFlattend",labelsFlattend)
  return f1_score(labelsFlattend, predictionArgmax, average='macro'),accuracy_score(labelsFlattend, predictionArgmax)

In [0]:
import random
import time 

def set_seed(seed,ngpu):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if ngpu > 0:
        torch.cuda.manual_seed_all(seed)
      
set_seed(42,torch.cuda.device_count())
#remove later

epochs=4
lossList=[]
max_grad_norm=1.0
for e in range(0, epochs):
    print("Start Epoch Number",(e + 1))
    print("Start Training")
    
    #Amount of time taken for training
    t1 = time.time()
    tr_loss, logging_loss = 0.0, 0.0
    model.train()
    tsteps=0
    for step, batch in enumerate(tdataloader):
        if step % 50 == 0 and not step == 0:
            print("Batch Completed  {:,}  of  {:,}.    Elapsed time is  {}".format(step, len(tdataloader),time.time() - t1))
        
        batch = tuple(t.to(device) for t in batch)
        inputs = {"input_ids": batch[0], "attention_mask": batch[1], "labels": batch[2]}
        model.zero_grad()        
        outputs = model(inputs["input_ids"],token_type_ids=None,attention_mask=inputs["attention_mask"], labels=inputs["labels"])

        loss = outputs[0]
        loss.backward()
        tr_loss += loss.item()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        tsteps+=1
        optimizer.step()
        sch.step()

    a_tr_loss = tr_loss /(tsteps)               
    lossList.append(a_tr_loss)
    
    print(" The training loss incured is  {0:.3f}".format(a_tr_loss))
    t2=time.time()
    print("  Training one epoch time taken",t2-t1)
    print(" Validation starts here ")
   
    t1 = time.time()
    model.eval()
    eval_loss = 0
    nb_eval_steps = 0
    eval_f1=0
    eval_acc=0

    for batch in tedataloader:       
        batch = tuple(t.to(device) for t in batch)        
        inputs = {"input_ids": batch[0], "attention_mask": batch[1], "labels": batch[2]}
        with torch.no_grad():        
            outputs = model(inputs["input_ids"],token_type_ids=None,attention_mask=inputs["attention_mask"])
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        label_ids = (inputs["labels"]).to('cpu').numpy()
        tmpf1score,tmpaccscore = calculateF1Score(logits, label_ids)
        eval_f1 = eval_f1+tmpf1score
        eval_acc=eval_acc+tmpaccscore
        nb_eval_steps += 1
        #print(" TEMP F1 score: {0:.3f}".format(tmpf1score))
        #print("TEMP  Accuracy score: {0:.3f}".format(tmpaccscore))


    print("  F1 score: {0:.3f}".format(eval_f1/nb_eval_steps))
    print("  Accuracy score: {0:.3f}".format(eval_acc/nb_eval_steps))
    
    t2=time.time()
    print("  Validating one epoch time taken ",t2-t1)
    
print("ALL DONE!!!")

Start Epoch Number 1
Start Training
Batch Completed  50  of  74.    Elapsed time is  38.96074342727661
 The training loss incured is  0.408
  Training one epoch time taken 57.37822198867798
 Validation starts here 
predictionArgmax [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
labelsFlattend [1 0 1 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
predictionArgmax [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
labelsFlattend [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0]
predictionArgmax [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
labelsFlattend [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
predictionArgmax [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
labelsFlattend [1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0]
predictionArgmax [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
labelsFlattend [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 